# Análise com o Modelo Multidimensional Cube

## Autores

| Nome | nUSP |
| :--- | :--- |
| Guilherme de Abreu Barreto | 12543033 |
| Lucas Eduardo Gulka Pulcinelli | 12547336 |
| Vinicio Yusuke Hayashibara | 13642797 |

## Introdução

O presente notebook demonstra a utilização da funcionalidade [Cube](https://www.postgresql.org/docs/current/cube.html) para exploração de um banco de dados construido com PostgreSQL sobre os dados do projeto [FAPESP COVID-19 DataSharing](https://repositoriodatasharingfapesp.uspdigital.usp.br/). Conforme enuncido para este exercício.

### Prérequisitos

Assume-se

- a instalação de todas as dependências do projeto;
- a existência de um banco de dados pré-configurado conforme explicitado em [notebook anterior](https://github.com/de-abreu/data-mining/blob/main/01-Prepara%C3%A7%C3%A3o%20de%20dados/Gera%C3%A7%C3%A3o%20de%20Databases%20com%20SQLalchemy%20e%20PostgreSQL.ipynb);
- A correta configuração do acesso ao banco de dados abaixo.

## Configuração

- DATABASE: O nome do database onde serão carregadas as informações. Atente-se se este não corresponde ao nome de um database preexistente ou que esteja sendo acessado, pois este será então sobrescrito.

- USER e PASSWORD: Informações de autententicação válidas e com privilégios para a criação de bancos de dados no servidor.

- HOST e PORT: A URL e porta para realização do acesso ao servidor.


In [1]:
DATABASE = "fapcov2103"
USER = "postgres"
PASSWORD = "password"
HOST = "localhost"
PORT = 5432

DATABASE_URI = f"postgresql+psycopg2://{USER}:{PASSWORD}@{HOST}/{DATABASE}"

## Importação das dependências

In [2]:
import sys
import pandas as pd
from PyQt6.QtWidgets import QApplication, QTableView, QMainWindow
from PyQt6.QtCore import QAbstractTableModel, Qt
from sqlalchemy import create_engine, select, func, text, MetaData, Table
from sqlalchemy.orm import declarative_base

## Enunciado

Listar a contagem de quantos Exames de Coronavírus foram feitos (procurar por ’corona’ ou ’covid’ em qualquer caixa de letra) por:

- Tipo de Exame
- Hospital em que o exame foi realizado
- Cidade de origem do paciente
- Quaisquer combinações dos fatores anteriores.

## Resolução

Todos os exames relacionados ao diagnóstico de Covid já foram separados na tabela AnalisesCovid do schema D2 no exercício anterior, então selecionamos esta e realizados a contagem dos subtotais e total usando Cube, como visto abaixo.

In [4]:
# Criamos a "engine", objeto pelo qual a conexão com o servidor é feita

engine = create_engine(
    DATABASE_URI,
    connect_args={'options': f'-c search_path=D2'},
)

# Criamos classes para representar as tabelas encontradas no servidor
# para representar as mesmas no modelo Object Relational Model (ORM)
# do SQLalchemy, carregando esta com metadados encontrados nas
# respectivas tabelas no servidor.

Base = declarative_base()
metadata = MetaData()

class ExamLab(Base):
    __table__ = Table("ExamLabs", metadata, autoload_with=engine)

class Paciente(Base):
    __table__ = Table("Pacientes", metadata, autoload_with=engine)


# Formulamos o statement pelo qual a seleção das colunas e aplicação
# do cube se dá. Designamos que todas as células a conter valores de
# marcação (placeholders), serão populadas com "~Todos". O caractere
# "~" garante que em uma ordenação alfabética estes valores figurarão
# por último.

placeholder="~Todos"

stmt = select(
    func.coalesce(ExamLab.DE_Exame, placeholder).label("Tipo de Exame"),
    func.coalesce(Paciente.CD_Municipio, placeholder).label("Cidade do Paciente"),
    func.coalesce(Paciente.DE_Hospital, placeholder).label("Hospital"),
    func.count().label("Numero de Exames")
).select_from(
    ExamLab.__table__.join(Paciente.__table__, ExamLab.ID_Paciente == Paciente.ID_Paciente)
).where(
    ExamLab.DE_Exame.op('~*')(r'(covid)|(sars.cov.2)|(corona)')
).group_by(
    func.cube(
        ExamLab.DE_Exame,
        Paciente.CD_Municipio,
        Paciente.DE_Hospital
    )
).order_by(
    "Tipo de Exame",
    "Cidade do Paciente",
    "Hospital"
)

# Carrega-se o resultado da consulta em um DataFrame.
df = pd.read_sql(stmt, engine)

### Primeiro resultado

Abaixo exibimos o primeiro concret obtido por esta consulta, e observamos haverem colunas duplicadas, como:

| Tipo de Exame | Cidade do Paciente | Hospital | Numero de Exames |
| ---: | ---: | ---: | ---: |
| Anticorpos IgA e IgG SARS-CoV-2/COVID19 | ~Todos | Einstein | 53 |
| Anticorpos IgA e IgG SARS-CoV-2/COVID19 | ~Todos | Einstein | 668 |
| Anticorpos IgA e IgG SARS-CoV-2/COVID19 |	~Todos | ~Todos | 53 |
| Anticorpos IgA e IgG SARS-CoV-2/COVID19 |	~Todos | ~Todos | 668 |

In [5]:
with pd.option_context('display.max_rows', None):
    display(df.style.hide(axis="index"))

Tipo de Exame,Cidade do Paciente,Hospital,Numero de Exames
Anticorpos IgA e IgG SARS-CoV-2/COVID19,BARUERI,Einstein,28
Anticorpos IgA e IgG SARS-CoV-2/COVID19,BARUERI,~Todos,28
Anticorpos IgA e IgG SARS-CoV-2/COVID19,COTIA,Einstein,14
Anticorpos IgA e IgG SARS-CoV-2/COVID19,COTIA,~Todos,14
Anticorpos IgA e IgG SARS-CoV-2/COVID19,EMBU,Einstein,2
Anticorpos IgA e IgG SARS-CoV-2/COVID19,EMBU,~Todos,2
Anticorpos IgA e IgG SARS-CoV-2/COVID19,GUARULHOS,Einstein,4
Anticorpos IgA e IgG SARS-CoV-2/COVID19,GUARULHOS,~Todos,4
Anticorpos IgA e IgG SARS-CoV-2/COVID19,JUNDIAI,Einstein,6
Anticorpos IgA e IgG SARS-CoV-2/COVID19,JUNDIAI,~Todos,6


## Discussão do resultado

Não conseguimos aferir a causa do surgimento desta duplicatas, mas conseguimos aferir que as contagens das duplicatas são sempre correspondentes ou inferiores ao valor obtido pela somatória das subcontagens dos hospitais.

In [9]:
grouping_cols = ["Tipo de Exame", "Cidade do Paciente", "Hospital"]
is_duplicate = df.duplicated(subset=grouping_cols, keep=False)
duplicated_rows = df[is_duplicate]

with pd.option_context('display.max_rows', None):
    display(duplicated_rows.style.hide(axis="index"))

Tipo de Exame,Cidade do Paciente,Hospital,Numero de Exames
Anticorpos IgA e IgG SARS-CoV-2/COVID19,~Todos,Einstein,53
Anticorpos IgA e IgG SARS-CoV-2/COVID19,~Todos,Einstein,668
Anticorpos IgA e IgG SARS-CoV-2/COVID19,~Todos,~Todos,53
Anticorpos IgA e IgG SARS-CoV-2/COVID19,~Todos,~Todos,668
Coronavírus 2019-nCoV,~Todos,HC,4550
Coronavírus 2019-nCoV,~Todos,HC,1904
Coronavírus 2019-nCoV,~Todos,~Todos,4550
Coronavírus 2019-nCoV,~Todos,~Todos,1904
Coronavirus Covid-19,~Todos,BPSP,7875
Coronavirus Covid-19,~Todos,BPSP,40595


Aplicamos uma técnica simples de ordenação descendente e removemos assim as duplicatas de menor valor. O resultado assim produzido é condizente com aquilo que esperávamos inicialmente.

In [14]:
# 1. Sort the DataFrame to bring the highest count to the top of each group
df_sorted = df.sort_values(by=grouping_cols + ["Numero de Exames"], ascending=False)

# 2. Drop duplicates, keeping only the first entry (which now has the highest count)
df_cleaned = df_sorted.drop_duplicates(subset=grouping_cols, keep='first')

# --- Display the cleaned result ---
with pd.option_context('display.max_rows', None):
    # Sort again for final presentation
    display(df_cleaned.sort_values(by=grouping_cols).style.hide(axis="index"))

Tipo de Exame,Cidade do Paciente,Hospital,Numero de Exames
anticorpos iga e igg sars-cov-2/covid19,BARUERI,Einstein,28
anticorpos iga e igg sars-cov-2/covid19,BARUERI,~Todos,28
anticorpos iga e igg sars-cov-2/covid19,COTIA,Einstein,14
anticorpos iga e igg sars-cov-2/covid19,COTIA,~Todos,14
anticorpos iga e igg sars-cov-2/covid19,EMBU,Einstein,2
anticorpos iga e igg sars-cov-2/covid19,EMBU,~Todos,2
anticorpos iga e igg sars-cov-2/covid19,GUARULHOS,Einstein,4
anticorpos iga e igg sars-cov-2/covid19,GUARULHOS,~Todos,4
anticorpos iga e igg sars-cov-2/covid19,JUNDIAI,Einstein,6
anticorpos iga e igg sars-cov-2/covid19,JUNDIAI,~Todos,6


Ainda assim, o resultado ao final é inconclusivo pois há uma variedade de 41 exames com nenhuma intersecção entre os Hospitais que os administraram. Cremos que isso se deve a estes usarem diferentes nomenclaturas para designar a um mesmo tipo de exame, isso quando não dão uma descrição demasiada genérica para o teste como "coronavirus covid-19". Assim o sendo, para obter um resultado informativo, estas classificações haveriam de ser verificadas com profissionais de saúde para se obter significativo agrupamento destes dados.